In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import scatter

import pathpyG as pp
pp.config['torch']['device'] = 'cpu'

In [2]:
def compute_weighted_outdegrees(graph):
    weighted_outdegree = scatter(graph.data.edge_weight, graph.data.edge_index[0], dim=0, dim_size=graph.data.num_nodes, reduce='sum')
    return weighted_outdegree

def compute_transition_probabilities(graph):
    weighted_outdegree = compute_weighted_outdegrees(graph)
    source_ids = graph.data.edge_index[0]
    return graph.data.edge_weight/ weighted_outdegree[source_ids]


In [3]:
from torch.utils.data import Dataset
import torch

# https://en.wikipedia.org/wiki/Pairing_function
# https://math.stackexchange.com/questions/1377929/generalization-of-cantor-pairing-function-to-triples-and-n-tuples
def cantor_pairing(x, y):
    """
    Computes the Cantor pairing value for two integers x and y.
    The Cantor pairing maps two integers to a unique integer.

    Args:
        x (int): The first integer.
        y (int): The second integer.

    Returns:
        int: The Cantor pairing value for the given integers x and y.
    """
    return (x + y) * (x + y + 1) // 2 + y

def cantor_encode_tensor(tnsr):
    """
    Encodes a list of integers in tensor rows into single integers using Cantor pairing.
    The function recursively applies the Cantor pairing function to pairs of elements
    in the input tensor until it encodes the entire list into a single integer.

    If the input tensor is empty, the function returns 0.

    Args:
        tnsr (torch.Tensor): A tensor containing a list of integers.

    Returns:
        torch.Tensor: The Cantor encoded integer representing the input list.
    """
    if tnsr.size(1) == 0:
        # Termination point. The added 0 has no effect on the returned integer
        return torch.tensor(0)
    else:
        return cantor_pairing(tnsr[:, 0], cantor_encode_tensor(tnsr[:, 1:]))


class WalksDataset(Dataset):
    """
    Dataset class to handle sequences of node walks.

    Args:
        dag_data (object): The input DAG data.
        dict_cantor_to_honode_ixs_mapping (dict): Dictionary mapping Cantor encoded indices to higher-order node indices.
        max_order (int): Maximum order of nodes in a walk.
    """

    def __init__(self, dag_data, dict_cantor_to_honode_ixs_mapping, max_order):
        self.max_order = max_order
        self.dict_cantor_to_honode_ixs_mapping = dict_cantor_to_honode_ixs_mapping
        self._preprocess_data(dag_data.dags)
        if max_order>1:
            self._create_walk_tensors()
            self._create_encoded_tensors()
        else:
            self._create_walk_tensors()
            self.bipartite_encoded_walks_by_length = self.walk_tensors_by_length



    def _preprocess_data(self, dags):
        """
        Preprocesses the DAG data to extract walks and their counts.

        Args:
            dags DAGData object
        """
        self.walks_by_length = {}
        self.walk_counts_by_length = {}
        self.total_sequences = 0
        
        for dag in dags:
            node_seq_path = dag.node_sequence.T[0]
            seq_length = len(node_seq_path)
            if seq_length not in self.walks_by_length:
                self.walks_by_length[seq_length] = []
                self.walk_counts_by_length[seq_length] = []
            self.walks_by_length[seq_length].append(node_seq_path)
            # Answer to Moritz question
            # probably one weight per DAG would be enough
            self.walk_counts_by_length[seq_length].append(int(dag.edge_weight.unique()))
            self.total_sequences += 1

    def _create_walk_tensors(self):
        """
        Creates tensors for each walk lengths.
        """
        self.walk_tensors_by_length = {
            length: torch.stack(walks, dim=0)
            for length, walks in self.walks_by_length.items()
        }
    def _create_encoded_tensors(self): # TODO: find better names for this method and the _bipartite_encode
        """
        Encode the walk representations using cantor encoding
        """
        self.bipartite_encoded_walks_by_length = {
            length: self._bipartite_encode(length)
            for length in self.walk_tensors_by_length
        }

    def _bipartite_encode(self, walk_length):
        """
        Encodes the walks in bipartite form, i.e.,
        representing transitions between indexes of higher-order nodes.

        In bipartite encoding, each transition in the walk sequence corresponds to a pair of indexes (from nth to n+1th column),
        where the first `self.max_order` transitions utilize indexes from the higher-order nodes of the i-th order tensors, 
        and the subsequent transitions use indexes of the `self.max_order` most recent nodes.

        Args:
            walk_length (int): The length of the walk.

        Returns:
            torch.Tensor: Bipartite-encoded walk sequences.
        """

        list_cantor_node_ixs_tensors = []
        
        for i in range(1, walk_length + 1):
            hon_ixs_tensor = self.walk_tensors_by_length[walk_length][:, max(0, i - self.max_order):i]
            cantor_encoded = cantor_encode_tensor(hon_ixs_tensor)
            mapped_indices = cantor_encoded.apply_(self.dict_cantor_to_honode_ixs_mapping[min(i, self.max_order)].get)
            list_cantor_node_ixs_tensors.append(mapped_indices)
            
        return torch.stack(list_cantor_node_ixs_tensors, dim=1)

    def __getitem__(self, index_tuple):
        """
        Retrieves a bipartite-encoded node sequence from the dataset.

        Args:
            index_tuple (tuple): A tuple containing the length of the sequence (int) and the index of the sequence within that length (int).

        Returns:
            torch.Tensor: The bipartite-encoded node sequence.
        """
        # This cannot work with batching. Could work if each walk length had its own Dataset object
        walk_length, index = index_tuple
        return self.bipartite_encoded_walks_by_length[walk_length][index]

    def __len__(self):
        """
        Returns the total number of sequences in the dataset.

        Returns:
            int: Total number of sequences.
        """
        return self.total_sequences


In [4]:
dag_data = pp.DAGData(pp.IndexMap(list("01234")))

dag_data.append_walk(list("0230230230230"), weight=300)
dag_data.append_walk(list("1241241241241"), weight=300)
dag_data.append_walk(list("0430241"), weight=1)
#
# dag_data.append_walk(list("0230230230230230230"), weight=30)


m = pp.MultiOrderModel.from_DAGs(dag_data, max_order=8)


In [5]:
def get_cantor_econding_mon_nodes(m):
    """ 
    Applies cantor encoding to the node sequences of the higher-order nodes for all the layers of a MultiOrderModel
    Returns a dictionary containing, for each layer, a mapping from the cantor index (encoding) of the sequence to the index of the ho-node in the Graph object
    """
    dict_cantor_to_honode_ixs_mapping = {}
    # This works cause the ho-node sequences are sorted by their indices
    for order, hon in m.layers.items():
        cantor_ids = cantor_encode_tensor(hon.data.node_sequence)
        cantor_to_node_ixs_mapping = {cantor_id.item(): i for i, cantor_id in enumerate(cantor_ids)}
        dict_cantor_to_honode_ixs_mapping[order] = cantor_to_node_ixs_mapping
        # print(order,hon.data.node_sequence.shape)
        # print(cantor_to_node_ixs_mapping)
    return dict_cantor_to_honode_ixs_mapping
dict_cantor_to_honode_ixs_mapping = get_cantor_econding_mon_nodes(m)

In [6]:
walk_data = WalksDataset(dag_data, dict_cantor_to_honode_ixs_mapping, max_order=5)
walk_data.bipartite_encoded_walks_by_length

{13: tensor([[0, 0, 0, 0, 0, 3, 5, 0, 3, 5, 0, 3, 5],
         [1, 2, 3, 3, 2, 4, 7, 2, 4, 7, 2, 4, 7]]),
 7: tensor([[0, 1, 2, 2, 1, 8, 6]])}

TODO: following cells contraints all this to be done on a single value of path length

In [7]:
# l = 7
# source_to_target_from_walks = walk_data.bipartite_encoded_walks_by_length[l]
# path_counts = walk_data.walk_counts_by_length[l]

In [8]:
# FIND ANOTHER WAY TO GET NODE COUNTS
from torch_geometric.loader import DataLoader
dag_graph = next(iter(DataLoader(dag_data.dags, batch_size=len(dag_data.dags)))).to(pp.config["torch"]["device"])


In [9]:
# unique_nodes, counts = torch.unique(dag_graph.node_sequence, return_counts=True)
# node_emission_probabilities = counts / counts.sum()

# source_to_target_edge_index_zeroth = torch.stack([
#     torch.zeros_like(source_to_target_from_walks[:, 0]),
#     source_to_target_from_walks[:, 0]
# ])

# # log likelihood
# tot_log_lh = 0
# # log likelihood for the 0-th steps
# lh_l = torch.mul(torch.log(node_emission_probabilities[source_to_target_edge_index_zeroth[1]]), torch.tensor(path_counts))
# tot_log_lh += lh_l.sum()

# for i in range(1, l - 1):
#     T = compute_transition_probabilities(m.layers[min(i + 1, walk_data.max_order)])
#     # Prepare source_to_target_edge_index
#     source_to_target_edge_index = source_to_target_from_walks[:, i:i + 2].T.squeeze()
#     # log likelihood for i-th steps
#     lh_l = torch.mul(torch.log(T[source_to_target_edge_index[1]]), torch.tensor(path_counts))
#     tot_log_lh += lh_l.sum()

# tot_log_lh


- Adapt to multiple walk lengths
- Better structure for package
- Degrees of freedom etc (actually making the likilhood ration test test)
    - This all thing above is getting lh (organize)
    - Need degrees of freedom
    - Need to actually perform che likelihood ratio test (see line 407 of this: https://github.com/IngoScholtes/pathpy/blob/master/pathpy/MultiOrderModel.py#L378). 
- test functions 
    - Dof computation
    - lh computation
    - encoding


In [10]:
def get_mon_likelihood(mon, walk_data, max_order = 1):
    assert max_order <= max(mon.layers), "max_order for the walk_data cannot be larger than the max_order in the mon"
    dict_cantor_to_honode_ixs = get_cantor_econding_mon_nodes(mon)
    inner_walk_data = WalksDataset(walk_data, dict_cantor_to_honode_ixs, max_order=max_order) #max_order=max(mon.layers))

    # zeroth order part:
    # TODO: FIND ANOTHER WAY TO GET NODE COUNTS for node emission probabilities ###########
    # notice that this is building the zeroth order of a MultiOrderModel
    from torch_geometric.loader import DataLoader
    dag_graph = next(iter(DataLoader(dag_data.dags, batch_size=len(walk_data.dags)))).to(pp.config["torch"]["device"])
    __unique_nodes, counts = torch.unique(dag_graph.node_sequence, return_counts=True) # check if the nodes' order is always ok
    node_emission_probabilities = counts / counts.sum()

    # Beginnign of log likelihood computation
    tot_log_lh = 0
    for walk_length in inner_walk_data.walks_by_length:
        # getting paths of length 'walk_length'
        bip_encoded_walks = inner_walk_data.bipartite_encoded_walks_by_length[walk_length]       
        path_counts = inner_walk_data.walk_counts_by_length[walk_length]
        # log likelihood for the 0-th order
        # necessary cause rigth now the 0th order is not in the mon
        target_indexes = bip_encoded_walks[:,0]
        lh_l = torch.mul(torch.log(node_emission_probabilities[target_indexes]), torch.tensor(path_counts))
        tot_log_lh += lh_l.sum()
        
        # Above we got the node probabilities; therefore, here we start from edge probability
        # Notice that this should also imply how the code above can be simplified
        for i in range(2, walk_length):
            if max_order>1:
                T = compute_transition_probabilities(m.layers[min(i-1,max_order-1)]) # NB: max_order -1 shouldn t be ok though
            else:
                T = node_emission_probabilities
            # Prepare source_to_target_edge_index
            target_indexes = bip_encoded_walks[:, i] 
            # log likelihood for i-th steps
            lh_l = torch.mul(torch.log(T[target_indexes]), torch.tensor(path_counts))
            tot_log_lh += lh_l.sum()
    return tot_log_lh.item()

# print(max(m.layers))
# cannot have that the max_order here is larger than the max_order in the mon
for o in range(max(m.layers)+1):
    print(o,get_mon_likelihood(m, dag_data, max_order=o))

0 -11541.60546875
1 -11541.60546875
2 -2651.2412109375
3 -1194.3135986328125
4 -1194.5257568359375
5 -1186.722900390625
6 -1186.722900390625
7 -1186.722900390625
8 -1186.722900390625


Probably there is somethign wrong in _bipartite_encode

In [12]:
def get_mon_dof(m, max_order=None, assumption="paths"):
    """
    The degrees of freedom fo the kth layer of a multi-order model this depende on the number of different paths of exactly length k in the graph.
    Therefore, we can obtain this values by summing the entries of the kth power of the binary adhacency matrix of the graph.
    Finally, we must consider that, due the conservation of probablility, all non-zero rows of the transition matrix of the higher-order network must sum to one. 
    This poses on additional constraint per row that respects the condition, which should be removed from the total count of degrees of freedom.

    Args:
        m (MultiOrderModel): The multi-order model.
        max_order (int, optional): The maximum order up to which model layers 
            shall be taken into account. Defaults to None, meaning it considers 
            all available layers.
        assumption (str, optional): If set to 'paths', only paths in the 
            first-order network topology will be considered for the degree of 
            freedom calculation. If set to 'ngrams', all possible n-grams will 
            be considered, independent of whether they are valid paths in the 
            first-order network or not. Defaults to 'paths'.

    Returns:
        int: The degrees of freedom for the multi-order model.

    Raises:
        AssertionError: If max_order is larger than the maximum order of 
            the multi-order network.
        ValueError: If the assumption is not 'paths' or 'ngrams'.
    """
    if max_order is None:
        max_order = max(m.layers)
    
    assert max_order <= max(m.layers), "Error: max_order cannot be larger than maximum order of multi-order network"

    dof = m.layers[1].data.num_nodes - 1  # Degrees of freedom for zeroth order

    if assumption == "paths":
        # COMPUTING CONTRIBUTION FROM NUM PATHS AND NONERO OUTDEGREES SEPARATELY
        # TODO: CAN IT BE DONE TOGETHER?

        # Adding dof from Number of paths of length k 
        for k in range(1, max_order + 1):
            
            if k == 1:
                edge_index = m.layers[1].data.edge_index
            else:
                edge_index = m.lift_order_edge_index(edge_index, num_len_k_paths)
            num_len_k_paths = edge_index.shape[1]  # Number of paths of length k
            dof += num_len_k_paths 
        
        # removing dof from total probability of nonzero degree nodes
        for k in range(1, max_order+1):
            
            if k == 1:
                edge_index_adj = m.layers[1].data.edge_index
                edge_index = edge_index_adj
            else:
                edge_index, _ = edge_index @ edge_index_adj
            num_nonzero_outdegrees = torch.unique(edge_index[0]).size(0)
            dof -=  num_nonzero_outdegrees

      
    elif assumption == "ngrams":
        for order in range(1, max_order + 1):
            dof += (m.layers[1].data.num_nodes ** order) * (m.layers[1].data.num_nodes - 1)
    else:
        raise ValueError(f"Unknown assumption {assumption} in input. The only accepted values are 'path' and 'ngram'")

    return int(dof)

for o in range(0,6):
    # get_mon_dof(m,max_order=o)
    print(get_mon_dof(m,max_order=o))

4
7
15
30
57
104


In [13]:
from scipy.stats import chi2
def lh_ratio_test(mon, walk_data, max_order_null = 0, max_order = 1, assumption='paths', significanceThreshold=0.01):
    assert max_order_null < max_order, 'Error: order of null hypothesis must be smaller than order of alternative hypothesis'
    assert max_order < max(mon.layers), f'Error: order of hypotheses ({max_order_null} and {max_order}) must be smaller than the maximum order of the MultiOrderModel {max(mon.layers)}'
    # let L0 be the likelihood for the null model and L1 be the likelihood for the alternative model

    # we first compute a test statistic x = -2 * log (L0/L1) = -2 * (log L0 - log L1)
    # get_mon_likelihood(mon, walk_data)
    x = -2 * (get_mon_likelihood(mon, walk_data, max_order=max_order_null) - get_mon_likelihood(mon, walk_data, max_order=max_order))

    # we calculate the additional degrees of freedom in the alternative model
    dof_diff = get_mon_dof(m,max_order, assumption = assumption) - get_mon_dof(m,max_order_null, assumption = assumption)
    print(x, dof_diff)

    # if the p-value is *below* the significance threshold, we reject the null hypothesis
    p = 1-chi2.cdf(x, dof_diff)
    return (p<significanceThreshold), p

Mess originates from the fact that mon and walk dataset can have incompatbile values. 

In [ ]:
# TODOS
# Deal with computation from 0th to 1st
# Do the zeroth and first order really have the same degrees of freedom in this example? Notice how it leads to a nan. 
lh_ratio_test(m, dag_data, max_order_null = 3, max_order = 4, assumption='paths', significanceThreshold=0.01)

tensor([[0, 0, 1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 7],
        [3, 4, 6, 7, 3, 4, 5, 6, 7, 0, 1, 2, 5]])
tensor([[ 0,  1,  1,  2,  3,  4,  5,  5,  6,  6,  7,  8,  9,  9, 10, 10, 11, 11,
         12, 12],
        [ 6,  7,  8, 11, 12,  6,  7,  8,  9, 10, 11, 12,  0,  1,  2,  3,  4,  5,
          9, 10]])
tensor([[ 0,  0,  1,  2,  3,  3,  4,  4,  5,  5,  6,  7,  8,  8,  9,  9, 10, 10,
         11, 11, 12, 13, 13, 14, 15, 16, 17, 17, 18, 18, 19, 19],
        [ 8,  9, 10, 11, 16, 17, 18, 19,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19,  0,  1,  2,  3,  4,  5,  6,  7, 12, 13, 14, 15]])
tensor([[0, 0, 1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 7],
        [3, 4, 6, 7, 3, 4, 5, 6, 7, 0, 1, 2, 5]])
tensor([[ 0,  1,  1,  2,  3,  4,  5,  5,  6,  6,  7,  8,  9,  9, 10, 10, 11, 11,
         12, 12],
        [ 6,  7,  8, 11, 12,  6,  7,  8,  9, 10, 11, 12,  0,  1,  2,  3,  4,  5,
          9, 10]])
-0.42431640625 12


(False, 1.0)

In [14]:
def estimate_order(mon, walk_data, max_order=None, significanceThreshold=0.01):
    """
    Selects the optimal maximum order of a multi-order network model for the
    observed paths, based on a likelihood ratio test with p-value threshold of p
    By default, all orders up to the maximum order of the multi-order model will be tested.

    @param paths: The path statistics for which to perform the order selection

    @param maxOrder: The maximum order up to which the multi-order model shall be tested.
    """
    if max_order == None:
        max_order = mon.max_order
    assert max_order <= max(mon.layers), 'Error: maxOrder cannot be larger than maximum order of multi-order network'
    assert max_order > 1, 'Error: maxOrder must be larger than one'

    max_accepted_order = 1

    # Test for highest order that passes
    # likelihood ratio test against null model
    for k in range(2, max_order+1):
        if lh_ratio_test(m, walk_data, max_order_null = k-1, max_order = k, significanceThreshold=significanceThreshold)[0]:
            max_accepted_order = k

    return max_accepted_order

estimate_order(m, dag_data, 6)

17780.728515625 8
2913.855224609375 15
-0.42431640625 27
15.605712890625 47
-0.0 76


3